## AI_A5
### Kasra Ghorbani // 810101489


#### Import required libraries

In [50]:
import pandas as pd
import string
import re
import nltk
from nltk.corpus import words
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import emoji
import numpy as np
from gensim.models import Word2Vec


#### Obtain set of english and stop words and punctuations

In [51]:
nltk.download('punkt')
nltk.download('words')
nltk.download('stopwords')
english_words = set(words.words())
stop_words = set(stopwords.words('english'))

[nltk_data] Error loading punkt: <urlopen error [WinError 10061] No
[nltk_data]     connection could be made because the target machine
[nltk_data]     actively refused it>
[nltk_data] Error loading words: <urlopen error [WinError 10061] No
[nltk_data]     connection could be made because the target machine
[nltk_data]     actively refused it>
[nltk_data] Error loading stopwords: <urlopen error [WinError 10061]
[nltk_data]     No connection could be made because the target machine
[nltk_data]     actively refused it>


#### Task 0

In [52]:

def preprocess_tweet(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove punctuations
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Remove links
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    text = re.sub(url_pattern, '', text)
    
    # Remove tagging usernames
    text = re.sub(r'@\w+', '', text)
    
    # Remove emojis
    text = emoji.replace_emoji(text, replace='')
    
    # Unify whitespaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Remove non-English words
    word_list = text.split()
    filtered_words = [word for word in word_list if word.lower() in english_words]
    text = ' '.join(filtered_words)
    
    # Tokenize text
    tokens = word_tokenize(text)
    
    # Remove stopwords
    tokens = [token for token in tokens if token.lower() not in stop_words]
    
    return tokens


#### Read CSV and Preprocess data

In [53]:
df = pd.read_csv("twitter-suicidal-data.csv")
df['tweet'] = df['tweet'].apply(preprocess_tweet)

#### Create Vectors for each token in the data

In [54]:
model = Word2Vec(sentences= df['tweet'], vector_size=300, window=5, min_count=1, sg=1)
matrix_list = []
max_matrix_tokens = 64
for i in range(len(df['tweet'])):
    matrix = []
    tokens = df['tweet'][i]
    for j in range(max_matrix_tokens):
        if j >= len(tokens):
            matrix.append(300*[0])
        else:
            if len(model.wv[tokens[j]]) != 0:
                matrix.append(model.wv[tokens[j]])
            else:
                matrix.append(300*[0])
    matrix_list.append(matrix)
    


In [55]:
import torch
print(torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import torch.nn as nn
import torch.nn.functional as F


class TweetClassifier(nn.Module):
    def __init__(self):
        super(TweetClassifier, self).__init__()
        # First set of convolutional layers
        self.conv1_3 = nn.Conv1d(in_channels=300, out_channels=64, kernel_size=3)
        self.conv1_5 = nn.Conv1d(in_channels=300, out_channels=64, kernel_size=5, padding=2)
        self.conv1_7 = nn.Conv1d(in_channels=300, out_channels=64, kernel_size=7, padding=3)
        
        # Second set of convolutional layers
        self.conv2_3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3)
        self.conv2_5 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5, padding=2)
        self.conv2_7 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=7, padding=3)
        
        # Fully connected layers
        self.fc1 = nn.Linear(3 * 128, 128)  # Adjusted the input size based on the architecture
        self.fc2 = nn.Linear(128, 2)  # Assuming binary classification

    def forward(self, x):
        # Apply first set of convolutional layers with ReLU activation
        x1 = F.relu(self.conv1_3(x))
        x2 = F.relu(self.conv1_5(x))
        x3 = F.relu(self.conv1_7(x))
        
        # Apply second set of convolutional layers with ReLU activation
        x1 = F.relu(self.conv2_3(x1))
        x2 = F.relu(self.conv2_5(x2))
        x3 = F.relu(self.conv2_7(x3))
        
        # Concatenate along the channel dimension
        x = torch.cat((x1, x2, x3), 2)  # Note the change in dimension for concatenation
        
        # Apply max pooling
        x = F.max_pool1d(x, kernel_size=2)
        
        # Flatten the output for the linear layer
        x = x.view(x.size(0), -1)
        
        # Apply linear layers with ReLU activation
        x = F.relu(self.fc1(x))
        x = self.fc2(x)  # No activation function here, it will be applied with the loss function
        
        return x

# Instantiate the model
model = TweetClassifier().to(device)


# Assuming 'matrix_list' is a list of tensors
tweet_tensors = torch.stack(matrix_list)
tweet_tensors = tweet_tensors.to(device)
# Forward pass (example for one batch)
output = model(tweet_tensors)


OSError: [WinError 126] The specified module could not be found. Error loading "C:\Users\a\AppData\Roaming\Python\Python311\site-packages\torch\lib\shm.dll" or one of its dependencies.